In [5]:
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
#import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.decomposition import SparsePCA
from sklearn.preprocessing import StandardScaler
import scipy.io
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
import math



sectors_list = ['Health Technology','Non-Energy Minerals','Transportation','Retail Trade','Electronic Technology','Distribution Services','Finance','Health Services','Industrial Services','Technology Services','Process Industries','Producer Manufacturing','Utilities','Consumer Services','Energy Minerals','Consumer Durables','Commercial Services','Consumer Non-Durables','Communications']

# sectors_list = ['Health Technology','Non-Energy Minerals','Transportation','Retail Trade']

#sectors_list = ['Health Technology']

profits = []
actions = []

# read in the prices of stocks
prices_stock_input = pd.read_csv('1000 stocks price 11_27.csv', header=0, index_col = 0, parse_dates=[0])
# read in the sector mapping of the stocks
sector_mapping = pd.read_csv('sector_mapping.csv', header = 0, index_col = False)

# drop the columns with NAs
prices_stock_input.dropna(axis='columns', inplace = True)
stock_names = list(prices_stock_input.columns.values)

# calculate of the returns of stocks
prices_shift = prices_stock_input[stock_names].shift(1)
returns_stock_input = (prices_stock_input[stock_names] - prices_shift)/prices_shift


In [6]:


for sector in sectors_list:
#sector = sectors_list[0]

    rolling_start_days = -2 # number of days before today
    previous_actions = pd.DataFrame([], columns = ['open_day', 'hedge_stock', 'action', 'stock','new_allocation$', 'new_quantity', 'open_price', 'signal', 'close_day','close_price', 'close_type'])
    s_last = []

    for testing_days in range(abs(rolling_start_days)+1):
        
        if testing_days == abs(rolling_start_days):
            last_day = True
        else:
            last_day = False

        # chose the according sector
        stocks_sector = list(sector_mapping[sector_mapping['Sector']==sector]['Stock'])

        prices_stock = prices_stock_input[stocks_sector].copy()
        returns_stock = returns_stock_input[stocks_sector].copy()

        history_days = 30
        rolling_start = len(returns_stock) + rolling_start_days # number of days before today

        # for rolling_day in range() 
        rolling_day = rolling_start + testing_days

        print 'sector ' + sector + 'day ' +  str(rolling_day)

        if rolling_day == rolling_start:
            first_day = True
        else:
            first_day = False

        if first_day:
            pre_df = pd.read_csv('open_positions_'+sector+'.csv', index_col = False, header = 0)
            pre_df = pre_df.rename(columns = {'day':'open_day', 'stocks':'stock', 'target_price':'open_price'})
            pre_df['open_day'] = 33
            pre_df['close_day'] = -99
            pre_df['close_price'] = -99
            pre_df['close_type'] = np.nan
            previous_actions = pd.concat([previous_actions, pre_df], axis = 0)

        prices_stock = prices_stock.iloc[rolling_day - history_days:rolling_day,].copy()
        returns_stock = returns_stock.iloc[rolling_day - history_days:rolling_day,].copy()


        number_days = len(returns_stock)
        number_stocks = returns_stock.shape[1]
        stock_names = list(prices_stock.columns.values)


        returns_na = returns_stock.isnull().sum()

        # Standardizing the features
        stock_corr = np.array(StandardScaler().fit_transform(returns_stock))

        # obtain the pricipal components
        num_components = 4
        #pca = SparsePCA(n_components=num_components, alpha = 2, ridge_alpha = 0.01, normalize_components=True)
        pca = PCA(n_components=num_components)
        stock_corr_pc = pca.fit_transform(stock_corr)
        stock_corr_pc_reduce = pd.DataFrame(data = stock_corr_pc, columns = ['pc' + str(i+1) for i in range(num_components)])

        # obtain the eigenvectors
        matrix_w = np.transpose(np.array(pca.components_) )

        beta_result = []
        residual_result = []
        alpha = []

        # for each stock, regression with the PCs and record coefs and residuals
        for i in range(number_stocks):
            # Create linear regression object
            regr = linear_model.LinearRegression()
            # Train the model using the training sets
            regr.fit(stock_corr_pc_reduce.values, stock_corr[:,i])
            beta_result.append(regr.coef_)
            alpha.append(regr.intercept_)
            residual_result.append(stock_corr[:,i] - regr.predict(stock_corr_pc_reduce.values))

        # calibrate based on residual    
        miu = []
        sigma = []
        s = []
        for i in range(number_stocks):
            miu_unit = np.mean(residual_result[i])
            miu.append(miu_unit)
            sigma_unit = np.std(residual_result[i])
            sigma.append(sigma_unit)

            #calculate s
            s_unit = []
            for j in range(number_days):
                # assume sigma is not zero
                s_unit.append( (residual_result[i][j] - miu[i])/sigma[i] )

            s.append(s_unit)


        s_last_out = [s[i][history_days-1] for i in range(len(s))]


        # set the constants
        s_bo=-1.96
        s_so=1.96  
        s_bc=-1.96
        s_sc=1.96
        k=0
        trades=0
        r=0;#0.03

        #cost
        t_cost=0;#2*pow(10, -4)
        borrowing_cost=0# %5

        #bool variables
        long=False
        short=False

        time_horizon = 60;
        number_of_stock = 30;
        regress_day=60;
        unwind_day=20;

        stop_lower=-4;
        stop_upper=4;

        returns_stock_std = returns_stock.std()

        result_by_day = []
        E = 100
        I = 100
        incremental_amt = 10000


        hedge = np.array(matrix_w)*np.array(beta_result)
        hedge = np.sum(hedge, axis = 1)


        for i in range(number_stocks):
            sigma_i = returns_stock_std[stock_names[i]]

            # buy to open
            if last_day:
                j = history_days - 1
                if(first_day is False and (s[i][j]-s_bo)<0 and (s_last[i]-s_bo)>0) or (first_day and (s[i][j]-s_bo)<0 ):
                    result_by_day.append([rolling_day+1, np.nan, 'buy to open', stock_names[i], incremental_amt, incremental_amt/prices_stock.iloc[j][i], prices_stock.iloc[j][i], s[i][j], -99, -99, np.nan])
                    for m in range(number_stocks):
                        sigma_m = returns_stock_std[stock_names[m]]
                        result_by_day.append([rolling_day+1, stock_names[i], 'short to hedge', stock_names[m], -(hedge[m]*sigma_i/sigma_m)*incremental_amt, -(hedge[m]*sigma_i/sigma_m)*incremental_amt/prices_stock.iloc[j][m], prices_stock.iloc[j][m], -99, -99, -99, np.nan])
                # sell to open
                elif(first_day is False and (s[i][j]-s_so)>0 and (s_last[i]-s_so)<0) or (first_day and (s[i][j]-s_so)>0 ):
                    result_by_day.append([rolling_day+1, np.nan, 'sell to open', stock_names[i],-incremental_amt, -incremental_amt/prices_stock.iloc[j][i], prices_stock.iloc[j][i], s[i][j], -99, -99, np.nan])
                    for m in range(number_stocks):
                        sigma_m = returns_stock_std[stock_names[m]]
                        result_by_day.append([rolling_day+1, stock_names[i], 'long to hedge', stock_names[m], hedge[m]*sigma_i/sigma_m*incremental_amt, hedge[m]*sigma_i/sigma_m*incremental_amt/prices_stock.iloc[j][m], prices_stock.iloc[j][m], -99, -99, -99, np.nan])

            # close long position        
            k = history_days - 1
            if(first_day is False and (s[i][k]-s_sc)>0 and (s_last[i]-s_sc)<0):
                # find the open day and information for the last buy to open stock
                previous_actions_buy_open = previous_actions[(previous_actions['action']=='buy to open') & (previous_actions['stock']==stock_names[i]) & (previous_actions['close_day']==-99)].sort_values(by = ['open_day'])
                # if there is an buy open stock: not empty and not closed
                if previous_actions_buy_open.empty is False:
                    # obtain all the buy to open positions
                    for index, row in previous_actions_buy_open.iterrows():
                        # stock open day
                        open_day = row['open_day']
                        # open stock name
                        hedge_stock = row['stock']

                        #print(open_day, hedge_stock, 'buy to open')

                        # mark the buy to open stock as close
                        previous_actions.loc[(previous_actions['action']=='buy to open') & (previous_actions['stock']==hedge_stock) & (previous_actions['open_day']==open_day), ['close_day']] = rolling_day+1  
                        previous_actions.loc[(previous_actions['action']=='buy to open') & (previous_actions['stock']==hedge_stock) & (previous_actions['open_day']==open_day), ['close_price']] = prices_stock.iloc[k][i] 
                        previous_actions.loc[(previous_actions['action']=='buy to open') & (previous_actions['stock']==hedge_stock) & (previous_actions['open_day']==open_day), ['close_type']] = 'sell to close' 

                        # close the short to hedge
                        short_hedge_names = np.array(previous_actions[(previous_actions['action']=='short to hedge') & (previous_actions['hedge_stock']==hedge_stock) & (previous_actions['open_day']==open_day)]['stock'])
                        for short_hedge_stock in short_hedge_names:
                            # mark the buy to open stock as close
                            previous_actions.loc[(previous_actions['action']=='short to hedge') & (previous_actions['hedge_stock']==hedge_stock) & (previous_actions['stock']==short_hedge_stock) & (previous_actions['open_day']==open_day), ['close_day']] = rolling_day+1  
                            previous_actions.loc[(previous_actions['action']=='short to hedge') & (previous_actions['hedge_stock']==hedge_stock) & (previous_actions['stock']==short_hedge_stock) & (previous_actions['open_day']==open_day), ['close_price']] = prices_stock.iloc[k,][short_hedge_stock] 
                            previous_actions.loc[(previous_actions['action']=='short to hedge') & (previous_actions['hedge_stock']==hedge_stock) & (previous_actions['stock']==short_hedge_stock) & (previous_actions['open_day']==open_day), ['close_type']] = 'close short hedge'
            elif(first_day is False and s[i][k]<stop_lower and s_last[i] >stop_lower):
                # find the open day and information for the last buy to open stock
                previous_actions_buy_open = previous_actions[(previous_actions['action']=='buy to open') & (previous_actions['stock']==stock_names[i]) & (previous_actions['close_day']==-99)].sort_values(by = ['open_day'])
                # if there is an buy open stock: not empty and not closed
                if previous_actions_buy_open.empty is False:
                    # obtain all the buy to open positions
                    for index, row in previous_actions_buy_open.iterrows():
                        # stock open day
                        open_day = row['open_day']
                        # open stock name
                        hedge_stock = row['stock']

                        #print(open_day, hedge_stock, 'buy to open')

                        # mark the buy to open stock as close
                        previous_actions.loc[(previous_actions['action']=='buy to open') & (previous_actions['stock']==hedge_stock) & (previous_actions['open_day']==open_day), ['close_day']] = rolling_day+1  
                        previous_actions.loc[(previous_actions['action']=='buy to open') & (previous_actions['stock']==hedge_stock) & (previous_actions['open_day']==open_day), ['close_price']] = prices_stock.iloc[k][i] 
                        previous_actions.loc[(previous_actions['action']=='buy to open') & (previous_actions['stock']==hedge_stock) & (previous_actions['open_day']==open_day), ['close_type']] = 'stop loss long position' 

                        # close the short to hedge
                        short_hedge_names = np.array(previous_actions[(previous_actions['action']=='short to hedge') & (previous_actions['hedge_stock']==hedge_stock) & (previous_actions['open_day']==open_day)]['stock'])
                        for short_hedge_stock in short_hedge_names:
                            # mark the buy to open stock as close
                            previous_actions.loc[(previous_actions['action']=='short to hedge') & (previous_actions['hedge_stock']==hedge_stock) & (previous_actions['stock']==short_hedge_stock) & (previous_actions['open_day']==open_day), ['close_day']] = rolling_day+1  
                            previous_actions.loc[(previous_actions['action']=='short to hedge') & (previous_actions['hedge_stock']==hedge_stock) & (previous_actions['stock']==short_hedge_stock) & (previous_actions['open_day']==open_day), ['close_price']] = prices_stock.iloc[k,][short_hedge_stock] 
                            previous_actions.loc[(previous_actions['action']=='short to hedge') & (previous_actions['hedge_stock']==hedge_stock) & (previous_actions['stock']==short_hedge_stock) & (previous_actions['open_day']==open_day), ['close_type']] = 'stop loss short hedge'

            #close short position
            k = history_days - 1
            if(first_day is False and (s[i][k]-s_bc)<0 and (s_last[i]-s_bc)>0):
                # find the open day and information for the last sell to open stock
                previous_actions_sell_open = previous_actions[(previous_actions['action']=='sell to open') & (previous_actions['stock']==stock_names[i]) & (previous_actions['close_day']==-99)].sort_values(by = ['open_day'])
                # if there is an sell to open stock: not empty and not closed
                if previous_actions_sell_open.empty is False:
                    # obtain all the sell to open positions
                    for index, row in previous_actions_sell_open.iterrows():
                        # stock open day
                        open_day = row['open_day']
                        # open stock name
                        hedge_stock = row['stock']

                        #print(open_day, hedge_stock, 'sell to open')

                        # mark the sell to open stock as close
                        previous_actions.loc[(previous_actions['action']=='sell to open') & (previous_actions['stock']==hedge_stock) & (previous_actions['open_day']==open_day), ['close_day']] = rolling_day+1  
                        previous_actions.loc[(previous_actions['action']=='sell to open') & (previous_actions['stock']==hedge_stock) & (previous_actions['open_day']==open_day), ['close_price']] = prices_stock.iloc[k][i] 
                        previous_actions.loc[(previous_actions['action']=='sell to open') & (previous_actions['stock']==hedge_stock) & (previous_actions['open_day']==open_day), ['close_type']] = 'close short position' 

                        # close the sell to hedge
                        long_hedge_names = np.array(previous_actions[(previous_actions['action']=='long to hedge') & (previous_actions['hedge_stock']==hedge_stock) & (previous_actions['open_day']==open_day)]['stock'])
                        for long_hedge_stock in long_hedge_names:
                            # mark the buy to open stock as close
                            previous_actions.loc[(previous_actions['action']=='long to hedge') & (previous_actions['hedge_stock']==hedge_stock) & (previous_actions['stock']==long_hedge_stock) & (previous_actions['open_day']==open_day), ['close_day']] = rolling_day+1  
                            previous_actions.loc[(previous_actions['action']=='long to hedge') & (previous_actions['hedge_stock']==hedge_stock) & (previous_actions['stock']==long_hedge_stock) & (previous_actions['open_day']==open_day), ['close_price']] = prices_stock.iloc[k,][long_hedge_stock] 
                            previous_actions.loc[(previous_actions['action']=='long to hedge') & (previous_actions['hedge_stock']==hedge_stock) & (previous_actions['stock']==long_hedge_stock) & (previous_actions['open_day']==open_day), ['close_type']] = 'close long hedge' 
            elif(first_day is False and s[i][k]>stop_upper and s_last[i]<stop_upper):
                # find the open day and information for the last sell to open stock
                previous_actions_sell_open = previous_actions[(previous_actions['action']=='sell to open') & (previous_actions['stock']==stock_names[i]) & (previous_actions['close_day']==-99)].sort_values(by = ['open_day'])
                # if there is an sell to open stock: not empty and not closed
                if previous_actions_sell_open.empty is False:
                    # obtain all the sell to open positions
                    for index, row in previous_actions_sell_open.iterrows():
                        # stock open day
                        open_day = row['open_day']
                        # open stock name
                        hedge_stock = row['stock']

                        #print(open_day, hedge_stock, 'sell to open')

                        # mark the sell to open stock as close
                        previous_actions.loc[(previous_actions['action']=='sell to open') & (previous_actions['stock']==hedge_stock) & (previous_actions['open_day']==open_day), ['close_day']] = rolling_day+1  
                        previous_actions.loc[(previous_actions['action']=='sell to open') & (previous_actions['stock']==hedge_stock) & (previous_actions['open_day']==open_day), ['close_price']] = prices_stock.iloc[k][i] 
                        previous_actions.loc[(previous_actions['action']=='sell to open') & (previous_actions['stock']==hedge_stock) & (previous_actions['open_day']==open_day), ['close_type']] = 'stop loss short position' 

                        # close the sell to hedge
                        long_hedge_names = np.array(previous_actions[(previous_actions['action']=='long to hedge') & (previous_actions['hedge_stock']==hedge_stock) & (previous_actions['open_day']==open_day)]['stock'])
                        for long_hedge_stock in long_hedge_names:
                            # mark the buy to open stock as close
                            previous_actions.loc[(previous_actions['action']=='long to hedge') & (previous_actions['hedge_stock']==hedge_stock) & (previous_actions['stock']==long_hedge_stock) & (previous_actions['open_day']==open_day), ['close_day']] = rolling_day+1  
                            previous_actions.loc[(previous_actions['action']=='long to hedge') & (previous_actions['hedge_stock']==hedge_stock) & (previous_actions['stock']==long_hedge_stock) & (previous_actions['open_day']==open_day), ['close_price']] = prices_stock.iloc[k,][long_hedge_stock] 
                            previous_actions.loc[(previous_actions['action']=='long to hedge') & (previous_actions['hedge_stock']==hedge_stock) & (previous_actions['stock']==long_hedge_stock) & (previous_actions['open_day']==open_day), ['close_type']] = 'stop loss long hedge'


        # after the day, keep the last day's actions and signal
        s_last = s_last_out
        result_by_day_df_new = pd.DataFrame(result_by_day, columns = ['open_day', 'hedge_stock', 'action', 'stock','new_allocation$', 'new_quantity', 'open_price', 'signal', 'close_day','close_price', 'close_type'])
        previous_actions = pd.concat([previous_actions, result_by_day_df_new], axis = 0)
        result_by_day_df = previous_actions

        if result_by_day_df.empty is False:
            # calculate current price
            current_price = pd.DataFrame({'stock': prices_stock.iloc[-1,:].reset_index().iloc[:,0], 'price':prices_stock.iloc[-1,:].reset_index().iloc[:,1]})
            # calculate long profit
            open_long_position = pd.merge(result_by_day_df[(result_by_day_df['close_day']==-99)& (result_by_day_df['new_quantity'] > 0 ) ].groupby(['stock'])['new_quantity'].sum().reset_index(), current_price, on = 'stock', how = 'left')
            open_long_position['position_value'] = open_long_position['new_quantity']*open_long_position['price']


            today = rolling_day + 1
            long_position_closed_today = pd.merge(result_by_day_df[(result_by_day_df['close_day']==today)& (result_by_day_df['new_quantity'] > 0 ) ].groupby(['stock'])['new_quantity'].sum().reset_index(),current_price, on = 'stock', how = 'left') 
            long_position_closed_today['position_value'] = long_position_closed_today['new_quantity']*long_position_closed_today['price']
            long_portfolio_value = open_long_position['position_value'].sum() + long_position_closed_today['position_value'].sum()

            newly_opened_long_position = pd.merge(result_by_day_df[(result_by_day_df['open_day']==today)& (result_by_day_df['new_quantity'] > 0 ) ].groupby(['stock'])['new_quantity'].sum().reset_index(), current_price, on = 'stock', how = 'left')
            newly_opened_long_position['position_value'] = newly_opened_long_position['new_quantity']*newly_opened_long_position['price']
            newly_opened_long_value = newly_opened_long_position['position_value'].sum()

            # calculate short profit
            open_short_position = pd.merge(result_by_day_df[(result_by_day_df['close_day']==-99)& (result_by_day_df['new_quantity'] < 0 ) ].groupby(['stock'])['new_quantity'].sum().reset_index(), current_price, on = 'stock', how = 'left')
            open_short_position['position_value'] = - open_short_position['new_quantity']*open_short_position['price']


            short_position_closed_today = pd.merge(result_by_day_df[(result_by_day_df['close_day']==today)& (result_by_day_df['new_quantity'] < 0 ) ].groupby(['stock'])['new_quantity'].sum().reset_index(),current_price, on = 'stock', how = 'left') 
            short_position_closed_today['position_value'] = - short_position_closed_today['new_quantity']*short_position_closed_today['price']
            short_portfolio_value = open_short_position['position_value'].sum() + short_position_closed_today['position_value'].sum()

            short_portfolio_value = open_short_position['position_value'].sum() + short_position_closed_today['position_value'].sum()

            newly_opened_short_position = pd.merge(result_by_day_df[(result_by_day_df['open_day']==today)& (result_by_day_df['new_quantity'] < 0 ) ].groupby(['stock'])['new_quantity'].sum().reset_index(), current_price, on = 'stock', how = 'left')
            newly_opened_short_position['position_value'] = - newly_opened_short_position['new_quantity']*newly_opened_short_position['price']
            newly_opened_short_value = newly_opened_short_position['position_value'].sum()

            profits.append([today, sector, long_portfolio_value, short_portfolio_value, newly_opened_long_value, newly_opened_short_value])

    # ouput the final actions
    previous_actions['sector'] = sector
    previous_actions.to_csv('result_'+sector+'_'+str(rolling_day)+'.csv', index = False)
    actions.append(previous_actions)


# write the combined actions to file
actions_df = pd.concat(actions, axis = 0)
actions_df.to_csv('back_testing_actions.csv', index = False)

# calculate the total profit
profits_df = pd.DataFrame(profits, columns = ['day', 'sector', 'long_portfolio_value', 'short_portfolio_value', 'newly_opened_long_value', 'newly_opened_short_value'])
profits_df = profits_df.sort_values(by = ['sector', 'day']).reset_index(drop = True)
profits_df['shift_long_portfolio_value'] = profits_df.groupby(['sector'])['long_portfolio_value'].shift(1)
profits_df['shift_short_portfolio_value'] = profits_df.groupby(['sector'])['short_portfolio_value'].shift(1)
profits_df['profit'] = profits_df['long_portfolio_value'] - profits_df['newly_opened_long_value'] - profits_df['shift_long_portfolio_value'] + profits_df['shift_short_portfolio_value']-profits_df['short_portfolio_value']+profits_df['newly_opened_short_value'] 
profits_df.to_csv('back_testing_profit.csv', index = False)

sector Health Technologyday 34
sector Health Technologyday 35
sector Health Technologyday 36
sector Non-Energy Mineralsday 34
sector Non-Energy Mineralsday 35
sector Non-Energy Mineralsday 36
sector Transportationday 34
sector Transportationday 35
sector Transportationday 36
sector Retail Tradeday 34
sector Retail Tradeday 35
sector Retail Tradeday 36
sector Electronic Technologyday 34
sector Electronic Technologyday 35
sector Electronic Technologyday 36
sector Distribution Servicesday 34
sector Distribution Servicesday 35
sector Distribution Servicesday 36
sector Financeday 34
sector Financeday 35
sector Financeday 36
sector Health Servicesday 34
sector Health Servicesday 35
sector Health Servicesday 36
sector Industrial Servicesday 34
sector Industrial Servicesday 35
sector Industrial Servicesday 36
sector Technology Servicesday 34
sector Technology Servicesday 35
sector Technology Servicesday 36
sector Process Industriesday 34
sector Process Industriesday 35
sector Process Industrie